Neste trabalho iremos fazer a busca dos melhores hiperparametros de uma SVM para Regressão num banco de dados em particular. Para o desenvolvimento utilizaremos a biblioteca sklearn.svm que apresenta a função  SVR() que implementa o regressor SVM e tem vários hiperparametros.
Vamos usar o kernel “rbf”, havendo 3 hiperparametros que consideramos como os mais importantes: C, gamma, e epsilon.

Vamos fazer a busca no range:
    - C entre 2^{-5} e 2^{15}  (uniforme nos expoentes);
    - gamma entre 2^{15} e 2^{-3} (uniforme nos expoentes);
    - episolon entre 0.05 e 10  (uniforme neste intervalo);
    
Utilizamos como dados de treino e testes os arquivos Xtreino5.npy e Xteste5.npy. Para as sáidas dos dados correspondenetres usamos os arquivos ytreino5.npy e yteste5.npy. Todos os arquivos foram disponibilizados na página no trabalho https://www.ic.unicamp.br/~wainer/cursos/1s2020/431/ex4.html.

### Importação das Bibliotecas

Como primeiro passo, importamos as bibliotecas necessárias para a implementação desse trabalho: numpy e sklearn.svm.

In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
import pyspark
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

from pyswarm import pso
from simanneal import Annealer
import optuna
from scipy.stats import loguniform
import random

### Leitura e Exibição dos arquivos

Após importamos as bibliotecas necessárias, realizamos a leitura dos arquivos .npy disponibilizados para uso neste trabalho. Para isso fizemos uso da função load() da biblioteca numpy. Armazenamos cada um dos arquivos de treino e de teste em variáveis locais.

In [2]:
x_treino = np.load('data/Xtreino5.npy')
x_teste = np.load('data/Xteste5.npy')
y_treino = np.load('data/ytreino5.npy')
y_teste = np.load('data/yteste5.npy')

### Medida de erro

Como primeira tarefa, para cada conjunto de hiperparametros, treinamos o SVM no conjunto de treino (Xtreino e ytreino), e medimos o erro absoluto médio (MAE) no conjunto de teste (Xteste e yteste).

## Randomized Search

In [3]:
n_combinations = 125

# Hyperparameters Search Space
# C_range = np.random.uniform(-5, 15, n_combinations).astype(float)
# C_range = 2**C_range

C_range = loguniform(2**-5, 2**15).rvs(size=n_combinations)

# gamma_range = np.random.uniform(-15, 3, n_combinations).astype(float)
# gamma_range = 2**gamma_range

gamma_range = loguniform(2**-15, 2** 3).rvs(size=n_combinations)

epsilon_range = np.random.uniform(0.05, 1.0, n_combinations).astype(float)

 
hyperparameters = {'gamma': list(gamma_range), 
                    'C': list(C_range),
                  'epsilon': list(epsilon_range)}
 
# print (hyperparameters)

In [4]:
# Run randomized search
randomCV = RandomizedSearchCV(SVR(kernel='rbf'), param_distributions=hyperparameters, n_iter=20)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']

print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))


The best performing C value is: 2635.22
The best performing gamma value is: 0.00003
The best performing epsilon value is:  0.15


In [5]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  2.745422696343669


### Grid seach

O próximo algoritimo de otimização proposto foi o Grid Search. Seguindo a especificação de uma busca em uma grid de 5x5x5, amostras do range de busca foram tomadas:

In [6]:
grid_c = random.sample(list(C_range), k=5)
grid_gamma = random.sample(list(gamma_range), k=5)
grid_episolon = random.sample(list(epsilon_range), k=5)

hyperparameters_grid = {'gamma': list(grid_gamma), 
                        'C': list(grid_c),
                        'epsilon': list(grid_episolon)}

In [7]:
# Run Grid Search
randomCV = GridSearchCV(SVR(kernel='rbf'), param_grid=hyperparameters_grid, cv = 5)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

The best performing gamma value is: 0.00010
The best performing C value is: 6398.23
The best performing epsilon value is:  0.93


In [8]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  2.498564684865281


### Otimização bayesiana

Outro algoritimo proposto foi a otimização bayesiana. Para a sua implementação utilziamos a biblioteca hyperopt, que dispnibiliza O regressor (TPE) para modelar a distribuição de probabilidades que é muito mais rápido que a implementação padrão utilizando “processos gaussianos”.

Esta implementação foi feita em passos:

### Definir a Função de mínimo

Como queremos pesquisar por Support Vector Machines (SVM), definimos um parâmetro params ['type'] como o nome do modelo, e uma função para executar o treinamento e retornar a precisão da validação cruzada. 
Como estamos tentando maximizar a precisão da validação cruzada, devemos negar esse valor para o hyperopt, pois o hyperopt sabe apenas como minimizar uma função.

In [9]:
def get_acc_status(clf,X_,y):
    acc = cross_val_score(clf, X_, y, cv=5).mean()
    return {'loss': -acc, 'status': STATUS_OK}

In [10]:
def objective(params):
    classifier_type = params['type']
    del params['type']
    if classifier_type == 'svm':
        clf = SVR(**params)
    else:
        return 0
    accuracy = cross_val_score(clf, x_treino, y_treino).mean()
    
    return {'loss': -accuracy, 'status': STATUS_OK}

### Definir espaço de pesquisa sobre os hiperparâmetros

In [11]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.uniform('C', (2**-5), (2**15)),
        'gamma': hp.uniform('gamma', (2**-15), (2**3)),
        'epsilon': hp.uniform('epsilon', 0.05, 1.0),
        'kernel': hp.choice('kernel', ['rbf'])
    },
])

### Selecionar um algoritimo de busca

As duas opções principais de algoritmos de busca são:

    - hyperopt.tpe.suggest: Estimadores da Árvore de Parzen, uma abordagem bayesiana que seleciona iterativa e adaptativamente novas configurações de hiperparâmetro para explorar com base em resultados anteriores;
    - hyperopt.rand.suggest: Pesquisa aleatória, uma abordagem não adaptativa que mostra o espaço de pesquisa.
    
Conforme pedido, utilziamos o algoritimo TPE.

### Executar o algoritmo de ajuste com hyperopt fmin ()

Definimos max_evals como o número máximo de pontos no espaço do hiperparâmetro para testar, ou seja, o número máximo de modelos para ajustar e avaliar.

O SparkTrials usa 2 argumentos:
    - parallelism: Número de modelos para ajustar e avaliar simultaneamente.
    - timeout: tempo máximo (em segundos) que fmin pode demorar. Este argumento é opcional.

O rastreamento automatizado do MLflow está ativado por padrão. Ligue para mlflow.start_run () antes de chamar fmin (), como mostra o exemplo abaixo.

In [12]:
hypopt_trials = Trials()
 
best_params = fmin(objective, search_space, algo=tpe.suggest, 
max_evals=125, trials= hypopt_trials)
 
best_gamma  = best_params['gamma']
best_C      = best_params['C']
best_epsilon= best_params['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

100%|██████████| 125/125 [00:11<00:00, 10.57trial/s, best loss: -0.4178449176776374]
The best performing gamma value is: 0.00965
The best performing C value is: 21142.37
The best performing epsilon value is:  0.30


In [13]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  3.9556840799481083


## PSO

In [14]:
# PARAMETERS
C_MIN = 2**(-5)
C_MAX = 2**15

GAMMA_MIN = 2**(-15)
GAMMA_MAX = 2**3

EPSILON_MIN = 0.05
EPSILON_MAX = 1.0

lb = np.array([C_MIN, GAMMA_MIN, EPSILON_MIN])
ub = np.array([C_MAX, GAMMA_MAX, EPSILON_MAX])

# FUNCTION
def svr_fun(X):
    c = X[0]
    g = X[1]
    eps = X[2]
    
    svr  = SVR(kernel='rbf', C=c, gamma=g, epsilon=eps)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae

print("PSO...")
x_opt, y_opt = pso(svr_fun, lb, ub, swarmsize=11, maxiter=11)

print(" C optimal: "+ str(x_opt[0])+
     "\n Gamma Optimal: "+ str(x_opt[1])+
     "\n Epsilon Optimal: "+ str(x_opt[2]))
print("MAE: ", str(y_opt))

PSO...
Stopping search: maximum iterations reached --> 11
 C optimal: 18603.56651115837
 Gamma Optimal: 3.0517578125e-05
 Epsilon Optimal: 0.05
MAE:  2.28368919281739


## Simulated Annealing

Para a implementação dos Métodos de Simmulate Annealing e CMA-ES foi utilizado a biblioteca optuna.

Para o método de Simulated Annealing a biblioteca fornece uma implementação via classe através dos chamados "samplers" para determinar os valores dos parâmetros a serem avaliados durante o teste. Para o Simmulated Annealing o sampler é implementado via classe explicitamente (vale ressaltar que estamos utilizando a implementação proposta pela própria documentação).

In [15]:
class SimulatedAnnealingSampler(optuna.samplers.BaseSampler):
    def __init__(self, temperature=100):
        self._rng = np.random.RandomState()
        self._temperature = temperature  # Current temperature.
        self._current_trial = None  # Current state.

    def sample_relative(self, study, trial, search_space):
        if search_space == {}:
            return {}

        #
        # An implementation of SA algorithm.
        #

        # Calculate transition probability.
        prev_trial = study.trials[-2]
        if self._current_trial is None or prev_trial.value <= self._current_trial.value:
            probability = 1.0
        else:
            probability = np.exp((self._current_trial.value - prev_trial.value) / self._temperature)
        self._temperature *= 0.9  # Decrease temperature.

        # Transit the current state if the previous result is accepted.
        if self._rng.uniform(0, 1) < probability:
            self._current_trial = prev_trial

        # Sample parameters from the neighborhood of the current point.
        #
        # The sampled parameters will be used during the next execution of
        # the objective function passed to the study.
        params = {}
        for param_name, param_distribution in search_space.items():
            if not isinstance(param_distribution, optuna.distributions.UniformDistribution):
                raise NotImplementedError('Only suggest_uniform() is supported')

            current_value = self._current_trial.params[param_name]
            width = (param_distribution.high - param_distribution.low) * 0.1
            neighbor_low = max(current_value - width, param_distribution.low)
            neighbor_high = min(current_value + width, param_distribution.high)
            params[param_name] = self._rng.uniform(neighbor_low, neighbor_high)

        return params

    #
    # The rest is boilerplate code and unrelated to SA algorithm.
    #
    def infer_relative_search_space(self, study, trial):
        return optuna.samplers.intersection_search_space(study)

    def sample_independent(self, study, trial, param_name, param_distribution):
        independent_sampler = optuna.samplers.RandomSampler()
        return independent_sampler.sample_independent(study, trial, param_name, param_distribution)





Por fim o é necessário criar uma função objetivo a qual deseja-se otimizar, em nosso caso procuramos minimizar o Erro Médio Absoluto (MAE) com uma validação no conjunto de testes da aplicação dos hyperparâmetros encontrados para o problema do SVM Regressor.

In [16]:
def objective(trial):
    c = trial.suggest_uniform('c', 2**(-5),  2**15)
    gamma = trial.suggest_uniform('gamma', 2**(-15),  2**3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae


sampler = SimulatedAnnealingSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=125)

[I 2020-05-06 08:05:52,891] Finished trial#0 with value: 5.759091794725524 with parameters: {'c': 4708.473217340531, 'gamma': 1.960995617457497, 'epsilon': 0.3753716179735884}. Best is trial#0 with value: 5.759091794725524.
[I 2020-05-06 08:05:52,965] Finished trial#1 with value: 5.760094554811656 with parameters: {'c': 6297.078408645507, 'gamma': 2.1723312887264545, 'epsilon': 0.40100220365343986}. Best is trial#0 with value: 5.759091794725524.
[I 2020-05-06 08:05:53,040] Finished trial#2 with value: 5.760858726602819 with parameters: {'c': 7827.4771810654165, 'gamma': 2.749376078704926, 'epsilon': 0.4134530108400437}. Best is trial#0 with value: 5.759091794725524.
[I 2020-05-06 08:05:53,116] Finished trial#3 with value: 5.762657707934435 with parameters: {'c': 8639.911480606264, 'gamma': 2.324129628294079, 'epsilon': 0.4747788674744218}. Best is trial#0 with value: 5.759091794725524.
[I 2020-05-06 08:05:53,188] Finished trial#4 with value: 5.766175797116519 with parameters: {'c': 104

[I 2020-05-06 08:05:55,600] Finished trial#35 with value: 5.759258245077275 with parameters: {'c': 9234.20948768915, 'gamma': 3.828063564718415, 'epsilon': 0.3497631572064879}. Best is trial#33 with value: 5.757551917435723.
[I 2020-05-06 08:05:55,679] Finished trial#36 with value: 5.758374158343062 with parameters: {'c': 9481.104144425222, 'gamma': 4.362503253148798, 'epsilon': 0.3168389918553539}. Best is trial#33 with value: 5.757551917435723.
[I 2020-05-06 08:05:55,757] Finished trial#37 with value: 5.755162533362653 with parameters: {'c': 7007.642442930221, 'gamma': 3.7338935787598904, 'epsilon': 0.22239205086372332}. Best is trial#37 with value: 5.755162533362653.
[I 2020-05-06 08:05:55,837] Finished trial#38 with value: 5.753390862283591 with parameters: {'c': 7365.575189962163, 'gamma': 3.9737538461516366, 'epsilon': 0.16266908209665187}. Best is trial#38 with value: 5.753390862283591.
[I 2020-05-06 08:05:55,916] Finished trial#39 with value: 5.7539899536441625 with parameters:

[I 2020-05-06 08:05:58,463] Finished trial#70 with value: 5.764763779657366 with parameters: {'c': 2136.848344759908, 'gamma': 3.420511341191799, 'epsilon': 0.5191946134520586}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:05:58,546] Finished trial#71 with value: 5.764586416859401 with parameters: {'c': 5026.751650624483, 'gamma': 4.117307742879868, 'epsilon': 0.5071246840823893}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:05:58,630] Finished trial#72 with value: 5.764966433875379 with parameters: {'c': 7335.518125805545, 'gamma': 3.5851811282103796, 'epsilon': 0.5230338423161766}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:05:58,714] Finished trial#73 with value: 5.76257599768707 with parameters: {'c': 7001.302490641241, 'gamma': 3.459195580676649, 'epsilon': 0.4553460894889067}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:05:58,798] Finished trial#74 with value: 5.761056769035683 with parameters: {'c

[I 2020-05-06 08:06:01,798] Finished trial#105 with value: 5.755078175144885 with parameters: {'c': 4538.2646420461115, 'gamma': 5.187509576115563, 'epsilon': 0.20758547201524108}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:06:01,917] Finished trial#106 with value: 5.7570637534581435 with parameters: {'c': 1614.254456790991, 'gamma': 5.86502874779658, 'epsilon': 0.26615971450078424}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:06:02,030] Finished trial#107 with value: 5.755514222123958 with parameters: {'c': 3269.1335050150146, 'gamma': 5.653314308406848, 'epsilon': 0.21804546104515035}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:06:02,128] Finished trial#108 with value: 5.7560765288333195 with parameters: {'c': 1870.006588012012, 'gamma': 6.328056350064079, 'epsilon': 0.23046856847440828}. Best is trial#59 with value: 5.749626135005567.
[I 2020-05-06 08:06:02,221] Finished trial#109 with value: 5.754356333331973 with par

Logo, como resultado os melhores parâmetros encontrados durante a busca são mostrados:

In [17]:
study.best_params

{'c': 6118.849781763269,
 'epsilon': 0.05211277484671926,
 'gamma': 2.9545932775065107}

Bem como o resultado para o  erro absoluto médio (MAE) no conjunto de teste com esses parâmetros:

In [18]:
study.best_value

5.749626135005567

## CMA-ES

Semelhantemente ao algoritmo de Simullate Annealing, a implementação do algoritmo de otimização CMA-ES também foi feita via biblioteca optunza. A implementação porém é encapsulada em um sampler default fornecido pela própria biblioteca, assim não sendo necessário a implementação de uma classe sampler específica, somente a função objetivo.

In [19]:
def objective(trial):
    c = trial.suggest_uniform('c', 2**(-5),  2**15)
    gamma = trial.suggest_uniform('gamma', 2**(-15),  2**3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae


sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=125)


[I 2020-05-06 08:06:03,980] Finished trial#0 with value: 5.768253211557609 with parameters: {'c': 19730.331330462675, 'gamma': 7.367101216080568, 'epsilon': 0.5925080948531264}. Best is trial#0 with value: 5.768253211557609.
[I 2020-05-06 08:06:04,049] Finished trial#1 with value: 5.766239864770038 with parameters: {'c': 16384.070924478205, 'gamma': 4.010463916482639, 'epsilon': 0.5540968651426552}. Best is trial#1 with value: 5.766239864770038.
[I 2020-05-06 08:06:04,123] Finished trial#2 with value: 5.766425348185869 with parameters: {'c': 16383.995977398801, 'gamma': 4.136751433873981, 'epsilon': 0.5580301977879147}. Best is trial#1 with value: 5.766239864770038.
[I 2020-05-06 08:06:04,196] Finished trial#3 with value: 5.7695564041419045 with parameters: {'c': 16383.973260185921, 'gamma': 3.988851387051228, 'epsilon': 0.6445597979188908}. Best is trial#1 with value: 5.766239864770038.
[I 2020-05-06 08:06:04,271] Finished trial#4 with value: 5.762866301303526 with parameters: {'c': 1

[I 2020-05-06 08:06:06,815] Finished trial#35 with value: 5.757527661015581 with parameters: {'c': 16384.17043001499, 'gamma': 4.128856191844779, 'epsilon': 0.29265697120871703}. Best is trial#31 with value: 5.750371968827088.
[I 2020-05-06 08:06:06,893] Finished trial#36 with value: 5.753062731165426 with parameters: {'c': 16384.328438196815, 'gamma': 4.351409964635339, 'epsilon': 0.14760729895084235}. Best is trial#31 with value: 5.750371968827088.
[I 2020-05-06 08:06:06,970] Finished trial#37 with value: 5.754318241732565 with parameters: {'c': 16384.205054605798, 'gamma': 4.1100436962159055, 'epsilon': 0.19093409715985693}. Best is trial#31 with value: 5.750371968827088.
[I 2020-05-06 08:06:07,045] Finished trial#38 with value: 5.760078804927224 with parameters: {'c': 16384.398701975715, 'gamma': 4.110270953895953, 'epsilon': 0.3717880610872234}. Best is trial#31 with value: 5.750371968827088.
[I 2020-05-06 08:06:07,121] Finished trial#39 with value: 5.752457340042541 with paramete

[I 2020-05-06 08:06:09,850] Finished trial#70 with value: 5.755590368180417 with parameters: {'c': 16383.90824354397, 'gamma': 3.9464623621687127, 'epsilon': 0.23364984959181878}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:09,946] Finished trial#71 with value: 5.750876977670059 with parameters: {'c': 16383.869318341425, 'gamma': 4.323783023945489, 'epsilon': 0.07670416720124403}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:10,031] Finished trial#72 with value: 5.751180312590264 with parameters: {'c': 16383.963243086013, 'gamma': 4.392617809624123, 'epsilon': 0.08581492378549457}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:10,115] Finished trial#73 with value: 5.754249899589772 with parameters: {'c': 16383.978843348274, 'gamma': 4.316315477106199, 'epsilon': 0.18649255974177958}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:10,199] Finished trial#74 with value: 5.75384125476031 with paramete

[I 2020-05-06 08:06:13,198] Finished trial#105 with value: 5.754877669084858 with parameters: {'c': 16384.042323241752, 'gamma': 4.837811665317978, 'epsilon': 0.20129646541241422}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:13,311] Finished trial#106 with value: 5.756155531256144 with parameters: {'c': 16384.35440593619, 'gamma': 4.815620157982458, 'epsilon': 0.24264710332727552}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:13,416] Finished trial#107 with value: 5.7524467611821875 with parameters: {'c': 16384.374787374956, 'gamma': 4.186110692563148, 'epsilon': 0.1291667473821231}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:13,528] Finished trial#108 with value: 5.752279834090349 with parameters: {'c': 16384.2217330922, 'gamma': 4.836791142588284, 'epsilon': 0.11693596078633679}. Best is trial#63 with value: 5.750131438152503.
[I 2020-05-06 08:06:13,635] Finished trial#109 with value: 5.751626807525203 with param

Logo, como resultado os melhores parâmetros encontrados durante a busca são mostrados:

In [20]:
study.best_params

{'c': 16384.08014158202,
 'epsilon': 0.05052406003703493,
 'gamma': 4.330758430702188}

Bem como o resultado para o  erro absoluto médio (MAE) no conjunto de teste com esses parâmetros:

In [21]:
study.best_value

5.750131438152503